# LIBRERIA sismo_utils

## 1. IMPORTANDO LIBRERIAS

In [2]:
import sys
import os
sys.path.append(os.getcwd()+'\..')
import pandas as pd
from lib import etabs_utils as etb
from lib import sismo_utils as sis
import sys
from ipywidgets import widgets
from lib import latex_utils as ltx
from IPython.display import clear_output, display

## 2. CONECTANDO A ETABS

Las siguientes lineas de codigo incializan la conexión con el modelo ETABS, donde la función etb.connect_to_etabs() returna dos resultados, el EtabsObject y el SapModel (en ese orden), solo sera necesario el segundo.
Adicionalmente se definiran las unidades con la función *etb.set_units(_SapModel,'Ton_m_C')*, siendo las unidades a elección del usuario segun la siguiente lista:
|Units|Valor numérico API ETABS|
|----|----|
|lb_in_F | 1|
|lb_ft_F|2|
|kip_in_F|3|
|kip_ft_F|4|
|kN_mm_C|5|
|kN_m_C |6|
|kgf_mm_C |7|
|kgf_m_C |8|
|N_mm_C	|9|
|N_m_C	|10|
|Ton_mm_C|11|
|Ton_m_C|12|
|kN_cm_C|13|
|kgf_cm_C|14|
|N_cm_C	|15|
|Ton_cm_C|16|

In [4]:
#Conexion con el modelo ETABS.
_,SapModel= etb.connect_to_etabs()

#Definir variables de salida 'Ton_m_C' o 'kgf_cm_C'
etb.set_units(SapModel,'Ton_m_C')

## 3. INTRODUCCIÓN DE INFORMACIÓN

Luego creamos las listas donde se contienen las informacion de los sistemas estructurales (sistema), las categorias de uso de edificación (categoria) y los casos de sismo para hacer las verificaciones sismicas (sis_loads) segun nos indica la norma E030.

In [5]:
sistemas = ['Pórticos de Concreto Armado',
            'Dual de Concreto Armado',
            'De Muros Estructurales de Concreto Armado',
            'Pórticos Especiales de Acero Resistentes a Momentos',
            'Pórticos Intermedios de Acero Resistentes a Momentos',
            'Pórticos Ordinarios de Acero Resistentes a Momentos',
            'Pórticos Especiales de Acero Concénticamente Arriostrados',
            'Pórticos Ordinarios de Acero Concénticamente Arriostrados',
            'Pórticos Acero Excéntricamente Arriostrados',
            'Muros de Ductilidad Limita de Concreto Armado',
            'Albañilería Armada o Confinada',
            'Madera']

categorias = ['A1 aislado',
              'A1 no aislado',
              'A2',
              'B',
              'C']

sis_loads = {'Sismo_EstX': 'Sx',
             'Sismo_EstY': 'Sy',
             'Sismo_DinX': 'SDx',
             'Sismo_DinY': 'SDy'}


En las siguientes lineas de codigo definimos la informacion de nuestro modelo en base a las listas creadas y el numero de pisos, sotanos y azoteas (estos ultimos para que no entren en las verificaciones de irregularidad)

In [6]:
zona = 4
suelo = 'S2'
sist_x = sistemas[0]
sist_y = sistemas[1]
categoria = categorias[4]
n_pisos = 4
n_sotanos = 0
n_azoteas = 0

Luego definimos nuestro objeto con la clase Sismo_e30()

In [7]:
sismo = sis.Sismo_e30()

Luego se registran los datos definidos lineas arriba para el modelo en el objeto segun la norma E030:

In [8]:
sismo.data.factor_zona(zona)
print('Zona sísmica: ',sismo.data.zona)
print('Factor de zona: ',sismo.data.Z)
sismo.data.factor_suelo(suelo)
print('Perfil de suelo: ',sismo.data.suelo)
print('Factor de suelo: ',sismo.data.S)
sismo.data.periodos_suelo()
print('Periodo de suelo Tp: ',sismo.data.Tp)
print('Periodo de suelo Tl: ',sismo.data.Tl)
sismo.data.sist_estructural(sist_x,sist_y)
print('Sistema estructural en dirección X: ',sismo.data.sistema_x)
print('Sistema estructural en dirección Y: ',sismo.data.sistema_y)
print('Coeficiente básico de reducción Ro en dirección X: ',sismo.data.Rox)
print('Coeficiente básico de reducción Ro en dirección Y: ',sismo.data.Roy)
print('Deriva máxima en dirección X: ',sismo.data.max_drift_x)
print('Deriva máxima en dirección Y: ',sismo.data.max_drift_y)
sismo.data.categoria_edificacion(categoria)
print('Categoría de la edificación: ',sismo.data.categoria)
print('Factor de uso: ',sismo.data.U)
sismo.data.set_pisos(n_pisos,n_azoteas,n_sotanos)
print('Número de pisos: ',sismo.data.n_pisos)
print('Número de azotea: ',sismo.data.n_azoteas)
print('Número de sótanos: ',sismo.data.n_sotanos)

Zona sísmica:  4
Factor de zona:  0.45
Perfil de suelo:  S2
Factor de suelo:  1.05
Periodo de suelo Tp:  0.6
Periodo de suelo Tl:  2.0
Sistema estructural en dirección X:  Pórticos de Concreto Armado
Sistema estructural en dirección Y:  Dual de Concreto Armado
Coeficiente básico de reducción Ro en dirección X:  8.0
Coeficiente básico de reducción Ro en dirección Y:  7.0
Deriva máxima en dirección X:  0.007
Deriva máxima en dirección Y:  0.007
Categoría de la edificación:  C
Factor de uso:  1.0
Número de pisos:  4
Número de azotea:  0
Número de sótanos:  0


Indicamos cuales son las irregularidades tanto en planta como en altura que suponemos que existen en la edificación:

In [9]:
sismo.data.irreg_altura(i_piso_blando=True,
                        i_piso_blando_e=False,
                        i_masa=False,
                        i_vertical=False,
                        i_discontinuidad_vertical=False,
                        i_discontinuidad_vertical_e=False)

print('Factor de Irregularidad en altura Ia: ',sismo.data.Ia)

sismo.data.irreg_planta(i_torsional=False,
                        i_torsional_e=True,
                        i_esquinas_entrantes=False,
                        i_discontinuidad_diafragma=False,
                        i_sistemas_no_paralelos=False)
print('Factor de Irregularidad en planta Ip: ',sismo.data.Ip)

Factor de Irregularidad en altura Ia:  0.75
Factor de Irregularidad en planta Ip:  0.6


A partir de los todos los datos ingresado calculamos los factores de reducción para dirección de análisis.

In [10]:
sismo.data.factor_R()
print('Factor de reducción en dirección X:',sismo.data.Rx)
print('Factor de reducción en dirección Y:',sismo.data.Ry)
print('Es regular?: {}'.format(sismo.data.is_regular))

Factor de reducción en dirección X: 3.5999999999999996
Factor de reducción en dirección Y: 3.15
Es regular?: False


Imprimimos todos los parametros que hemos introducido que el programa defina a partir de estos teniendo en cuenta la norma E.030

In [11]:
sismo.data.show_params()


 Parámetros de sitio:
Factor de zona: 
    Z=0.45
Factor de Importancia: 
    U=1.00
Factor de Suelo: 
    S=1.05
Periodos del Suelo: 
    Tp=0.60
    Tl=2.00
Factor Básico de Reducción:
    Rox=8.00
    Roy=7.00
Irregularidad en planta:
    Ip=0.60
Irregularidad en altura:
    Ia=0.75
Factor de Reducción:
    Rx=3.60
    Ry=3.15



Introducimos los casos de carga definidos previamente para obtener las tablas necesaria para verificar las irregularidades

In [12]:
sismo.loads.set_seism_loads(sis_loads)

Con todo esto ya tenemos definido todos los datos para empezar a verificar las irregularidades.

## 4. VERIFICACIÓN DE IRREGULARIDADES

Antes de verificar las irregularidades segun la NTE E030 debemos de obtener los datos necesario del modelo ETABS. Primero indicar que el modelo debe estar corrido indicando que se debe tener activado el cálculo del centro de rigidez del diafragma en el modelo antes de iniciar el análisis (revisar el cuadro "Analyze->Set Load Cases To Run"). Si el codigo presentara error.

### 4.1 SISMO ESTÁTICO

Esta seccion comprende de 2 funciones: ana_modal(SapModel,report=False) y el sismo_estatico(SapModel,report=False). El primero obtiene la tabla de resultados del analisis modal y el segundo define todos los factores y el coeficiente ZUCS/R de la norma NTE E0.30. Recordar que estas funciones estan definidas para la clase Sismo_e30 y que anteriormente fue denominada "sismo" a nuestro objeto en estudio (sismo = sis.Sismo_e30()) 

### FUNCIÓN: ana_modal(self,SapModel,report=False)

La finalidad de esta función es la de obtener a partir de la tabla de resultado del análisis modal ("Modal Participating Mass Ratios") los periodos fundamentales para cada dirección de análisis y la tabla con las masas participativas en UX,UY y RZ, con sus respectivos valores acumulados SumUX,SumUY y SumRZ, con el fin de verificar que si se obtiene al menos el 90% segun lo indicado en la sección 29.1.2 de la NTE E0.30. Caso contrario solo se indica un mensaje que pide aumentar los grados de libertad pero aun sigue definiendo las tablas y los periodos
Adicionalmente da la opcion de generar un reporte (report=False, por defecto)

In [13]:
help(sismo.ana_modal)

Help on method ana_modal in module lib.sismo_utils:

ana_modal(SapModel, report=False) method of lib.sismo_utils.Sismo_e30 instance
    Devuelve datos del analisis modal, y los periodos fundamentales
    para cada dirección de análisis si la masa participativa es mayor al
    90%. Revise 29.1.2 NTE 030 2020
    
    Parámetros
    ----------
    SapModel: objeto propio del ETABS
    
    Returns
    -------
    mensaje: ----Aumentar grados de libertad, si MP_y o MP_x<0.9
    modal=tabla "Modal Participating Mass Ratios"
    T_x=Periodo fundamental en la dirección x
    T_y=Periodo fundamental en la dirección y



In [14]:
sismo.ana_modal(SapModel,True)


Análisis Modal:
Masa Participativa X: 1.00
Masa Participativa Y: 1.00
Periodo y Masa Participativa X: Tx=0.360, Ux=0.864
Periodo y Masa Participativa Y: Ty=0.273, Uy=0.872


Los valores de los periodos fundamentales de vibracion para cada dirección de análisis.

In [15]:
print(sismo.data.Tx)
print(sismo.data.Ty)

0.36
0.273


La tabla que genera esta funcion es:

In [17]:
display(sismo.tables.modal)

,Mode,Period,UX,UY,RZ,SumUX,SumUY,SumRZ
0,1,0.36,0.864000,0.000000,2.486E-06,0.864,0,2.486E-06
1,2,0.273,0.000000,0.871800,1.513E-06,0.864,0.8718,4E-06
2,3,0.225,0.000011,0.000001,0.8499,0.864,0.8718,0.8499
3,4,0.101,0.118800,0.000000,4.188E-06,0.9828,0.8718,0.8499
4,5,0.077,0.000000,0.111900,0,0.9828,0.9837,0.8499
5,6,0.062,0.000000,0.000000,0.1317,0.9828,0.9837,0.9816
6,7,0.048,0.017200,0.000000,6.349E-07,1,0.9837,0.9816
7,8,0.037,0.000000,0.016300,0,1,1,0.9816
8,9,0.029,0.000002,0.000000,0.0184,1,1,1


### FUNCIÓN: sismo_estatico(self,SapModel,report=False)

La finalidad de esta funcion es la de definir los valores de los exponentes en altura en ambas direcciones de analisis (kx y ky), defenir los valores de los factores de amplificaciones dinamica (Cx y Cy) y los coeficientes de sismo estatico ZUCS/Rx ZUCS/Ry. 
Adicionalmente da la opcion de generar un reporte (report=False, por defecto)


In [18]:
help(sismo.sismo_estatico)

Help on method sismo_estatico in module lib.sismo_utils:

sismo_estatico(SapModel, report=False) method of lib.sismo_utils.Sismo_e30 instance
    Registra en un diccionario: el factor de Reduccion total R, los factores de 
    amplificacion sísmica en ambas direcciones de analisis, los exponentes de altura 
    relacionados con los periodos fundamentales en cada direccion de analisis y los 
    coeficientes de sismo estatico para cada direccion de analisis.
    
    Parámetros
    ----------
    SapModel: objeto propio del ETABS
    N: Numero de pisos
    Z: Factor de zona
    U: Factor de uso
    S: Factor de amplificacion del suelo
    Tp: Periodo del suelo para C=cte
    Tl: Periodo del suelo para desplazamientos constantes
    Ip: Factor de irregularidad en planta
    Ia: Factor de irregularidad en altura
    R_o: Coeficiente basico de reduccion
    
    Returns
    -------
    Imprime los valores de 
    data: Diccionario que contiene la tabla "Modal Participating Mass Ratios", k_

Este método tiene integrado 3 funciones adicionales: get_k(T), get_C(T) y get_ZUCS_R(C,R) para calcular el exponente relacionado con el periodo, el factor de amplificacion sismica, y el coeficiente sismico respectivamente.

In [19]:
sismo.sismo_estatico(SapModel,report=True)

Factor de Reduccion con Irregularidades en X: R=3.5999999999999996
Factor de Reduccion con Irregularidades en Y: R=3.15
C en X: 2.50
C en Y: 2.50

Coeficiente de sismo estático X: 0.328
Coeficiente de sismo estático Y: 0.328
Exponente de altura X: 1.00
Exponente de altura Y: 1.00


### 4.2 REVISIÓN POR TORSIÓN

Esta seccion esta comprendida por la función: *irregularidad_torsion(self,SapModel)*, 

### FUNCIÓN: irregularidad_torsion(self,SapModel)

La finalidad de esta función es hacer la verificación de irregularidad torsional de la edificación. Empieza por leer los datos de la tabla de resultados "Story Max Over Avg Drifts" del modelo ETABS, ordena esta tabla para una mejor visualización y hace la comparacion de los datos para ver si cumple con los requisitos de la tabla N°9 de la NTE E0.30. Termina por definir dicha tabla de resultados.

In [20]:
help(sismo.irregularidad_torsion)

Help on method irregularidad_torsion in module lib.sismo_utils:

irregularidad_torsion(SapModel) method of lib.sismo_utils.Sismo_e30 instance
    Registra en un DataFrame la tabla de derivas maximas de entrepiso, 
    deriva promedio y la razon de derivas maximas sobre derivas promedios
    
    Parámetros
    ----------
    SapModel: objeto propio del ETABS
    loads: Casos de carga
    max_drift: deriva máxima
    R: coeficiente de reduccion sismica
    is_regular: boleano, true si es regular, false si es irregular
    
    Returns
    -------
    table: DataFrame que contiene la tabla "Story Max Over Avg Drifts", Max Drift,
    Avg Drift y Ratio



In [21]:
sismo.irregularidad_torsion(SapModel)

In [22]:
display(sismo.tables.torsion_table)

,Story,OutputCase,Direction,Max Drift,Avg Drift,Ratio,Height,Drifts,Drift < Dmax/2,tor_reg
0,Story3,Sx,X,0.004506,0.004149,1.086,3.6,0.016222,False,Regular
1,Story3,Sy,Y,0.001968,0.001916,1.027,3.6,0.006199,False,Regular
2,Story3,SDx Max,X,0.004028,0.003867,1.042,3.6,0.014501,False,Regular
3,Story3,SDx Max,Y,0.000643,0.000622,1.034,3.6,0.002315,True,Regular
4,Story3,SDy Max,X,0.001227,0.00117,1.049,3.6,0.004417,False,Regular
5,Story3,SDy Max,Y,0.00176,0.001751,1.005,3.6,0.006336,False,Regular
6,Story2,Sx,X,0.005596,0.005195,1.077,3.6,0.020146,False,Regular
7,Story2,Sy,Y,0.002598,0.002536,1.024,3.6,0.008184,False,Regular
8,Story2,SDx Max,X,0.004952,0.004773,1.037,3.6,0.017827,False,Regular
9,Story2,SDx Max,Y,0.000826,0.000802,1.03,3.6,0.002974,True,Regular


### 4.3 REVISIÓN POR PISO BLANDO

Esta seccion esta comprendida por la función: *piso_blando(self,SapModel)* 

### FUNCIÓN: piso_blando(self,SapModel)

La finalidad de esta función es hacer la verificación de la irregularidad en altura de piso blando o irregularidad de Rigidez segun los requisitos de la Tabla N°8 de la norma NTE E0.30. Lo que hace es leer los casos de sismo estatico y dinámicos definidos en el modelo y que fueron definidos en la lista se de sis_loads, luego obtiene las tablas de 'Diaphragm Center Of Mass Displacements' y 'Story Forces' las cuales las modifica para obtener los desplazamientos relativos de entrepiso y con ello la rigidez lateral de entrepiso, tal como se indica en la norma NTE E0.30, luego procede a hacer las verificaciones (70% de la rigidez lateral del entrepiso inmediato superior y 80% de la rigidez lateral promedio de los tres niveles superiores adyacentes). Termina por definir la tabla *self.tables.piso_blando_table* donde esta indicado dicho procedimiento.

In [23]:
help(sismo.piso_blando)

Help on method piso_blando in module lib.sismo_utils:

piso_blando(SapModel) method of lib.sismo_utils.Sismo_e30 instance



In [24]:
sismo.piso_blando(SapModel)

In [25]:
display(sismo.tables.piso_blando_table)

,Story,OutputCase,UX,UY,ΔUX,ΔUY,VX,VY,lat_rig(k),0.7_prev_k,0.8k_prom,is_reg
0,Story3,Sy,-3E-06,0.007068,-0.000005,1.916000e-03,0,-34.4361,17972.912317,0.000000,0.0,Regular
1,Story2,Sy,2E-06,0.005152,0.000001,2.537000e-03,0,-71.0163,27992.234923,12581.038622,0.0,Regular
2,Story1,Sy,1E-06,0.002615,0.000001,2.615000e-03,0,-93.4545,35737.858509,19594.564446,0.0,Regular
3,Story3,SDx Max,0.012806,0.001874,0.003676,5.090000e-04,37.5374,9.565,18791.748527,0.000000,0.0,Regular
4,Story2,SDx Max,0.00913,0.001365,0.004582,6.740000e-04,74.2772,19.1643,28433.679525,13154.223969,0.0,Regular
5,Story1,SDx Max,0.004548,0.000691,0.004548,6.910000e-04,95.2085,24.6712,35703.617945,19903.575668,0.0,Regular
6,Story3,Sx,0.014536,0,0.004159,1.000000e-06,-40.1777,0,9660.423179,0.000000,0.0,Regular
7,Story2,Sx,0.010377,-1E-06,0.005187,-6.553000e-07,-82.857,0,15973.973395,6762.296225,0.0,Regular
8,Story1,Sx,0.00519,-3.447E-07,0.005190,-3.447000e-07,-109.0364,0,21008.940270,11181.781377,0.0,Regular
9,Story3,SDy Max,0.003842,0.006248,0.001103,1.699000e-03,11.2612,31.8832,18765.862272,0.000000,0.0,Regular


### 4.4 REVISIÓN POR MASA O PESO

Esta seccion esta comprendida por la función: *irregularidad_masa(self,SapModel)* 

### FUNCIÓN: irregularidad_masa(self,SapModel)

La finalidad de esta función es hacer la verificación de la irregularidad en altura de masa o peso, para ello obtiene la tabla 'Mass Summary by Story' del modelo ETABS, define los pisos (excepto azoteas y sótanos) y compara la masa en cada piso. Si la masa supera 1.5 veces el masa del piso superior o del inferior indica que es irregular.

In [26]:
help(sismo.irregularidad_masa)

Help on method irregularidad_masa in module lib.sismo_utils:

irregularidad_masa(SapModel) method of lib.sismo_utils.Sismo_e30 instance



In [27]:
sismo.irregularidad_masa(SapModel)

In [28]:
sismo.tables.rev_masa_table

,Story,Mass,1.5 Mass,story_type,is_regular
0,Story3,9.19054,,Azotea,Regular
1,Story2,13.84954,20.77431,Piso,Regular
2,Story1,14.61188,21.91782,Piso,Regular
3,Base,2.72264,,Sotano,Regular


### 4.5 REVISION DE DERIVAS DE ENTREPISO

Esta seccion esta comprendida por la función: *derivas(self)* 

### FUNCIÓN: derivas(self):

La finalidad de esta función es hacer la verificación de las derivas de entrepiso maxima con la maxima deriva permitida según la tabla N°11 de la NTE E0.30. Se obtiene la tabla con las maximas derivas de entrepiso y se hace la comparación con la maxima deriva de entrepiso.

In [29]:
help(sismo.derivas)

Help on method derivas in module lib.sismo_utils:

derivas() method of lib.sismo_utils.Sismo_e30 instance
    # Derivas



In [30]:
sismo.derivas()

In [32]:
display(sismo.tables.drift_table)

,Story,OutputCase,Direction,Drifts,Drift_Check
0,Story3,Sx,X,0.016222,No Cumple
1,Story3,Sy,Y,0.006199,Cumple
2,Story3,SDx Max,X,0.014501,No Cumple
3,Story3,SDx Max,Y,0.002315,Cumple
4,Story3,SDy Max,X,0.004417,Cumple
5,Story3,SDy Max,Y,0.006336,Cumple
6,Story2,Sx,X,0.020146,No Cumple
7,Story2,Sy,Y,0.008184,No Cumple
8,Story2,SDx Max,X,0.017827,No Cumple
9,Story2,SDx Max,Y,0.002974,Cumple


### 4.6 CENTRO DE MASAS Y RIGIDECES

Esta seccion esta comprendida por la función: *derivas(self)* 

### FUNCIÓN: centro_masa_inercia(self,SapModel):

La finalidad de esta función es obtener los centros de masa y rigidez por entrepiso extrayendo esta informacion de la tabla "Centers Of Mass And Rigidity" del modelo ETABS.

In [33]:
help(sismo.centro_masa_inercia)

Help on method centro_masa_inercia in module lib.sismo_utils:

centro_masa_inercia(SapModel) method of lib.sismo_utils.Sismo_e30 instance



In [34]:
sismo.centro_masa_inercia(SapModel)

In [35]:
display(sismo.tables.CM_CR_table)

,Story,XCCM,XCR,YCCM,YCR,DifX,DifY
0,Story3,3.135,3.135,7.4157,7.3571,0.0000,0.0586
1,Story2,3.1395,3.135,7.3503,7.3753,0.0045,-0.0250
2,Story1,3.1405,3.135,7.3345,7.3975,0.0055,-0.0630


### 4.7 VERIFICACIÓN DE LA CORTANTE BASAL

Esta seccion esta comprendida por la función: *min_shear(self,SapModel,story='Story1')*

### FUNCIÓN: min_shear(self,SapModel,story='Story1')

La finalidad de esta función es verificar la cortante basal que se obtiene de los casos de carga dinamicos de sismo con los casos de carga estatico y aplicar el criterio de la sección de Fuerza Cortante Minima de la seccion 29.4 de la NTE E0.30. El parametro 'story' es por defecto el 'Story1'.

In [36]:
help(sismo.min_shear)

Help on method min_shear in module lib.sismo_utils:

min_shear(SapModel, story='Story1') method of lib.sismo_utils.Sismo_e30 instance



In [37]:
sismo.min_shear(SapModel,story='Story1')

In [38]:
display(sismo.tables.shear_table)

,0,1,2
0,,X,Y
1,V din (Ton),95.2085,82.2373
2,V est (Ton),-109.0364,-93.4545
3,% min,90,90
4,%,87.32,88.0
5,F.E.,1.03,1.02


### 4.8 ANALISIS SISMICO

Esta seccion esta comprendida por la función: *analisis_sismo(self, SapModel,base_story='Story1')*

### FUNCIÓN: analisis_sismo(self, SapModel,base_story='Story1')

La finalidad de esta función es realizar todas la verificaciones del analisis sismico llamando a todas las funciones previamente mencionadas que se relacionan a alguna verificación.

In [40]:
#sismo.analisis_sismo(SapModel)

tablas = sismo.tables
#sismo.generate_memoria()